In [ ]:
"""
This code has been made to calculate a rating curve for a river given its water discharge data Q_w (m/s) and suspended sediment
concentration data Q_s (mg/L). The rating curve is based on the following equation:

a*Q_s**b

Where "a" and "b" are calculated regression coefficients. The code assumes the read-in file is a .csv file from a USGS dataset. Q_w vs Q_s 
datapoints should share a date- if this data was not collected on the same date, additional datetime interpolation may be required.
"""

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
#Read in your file:
River = pd.read_csv('../Data/RiverData.csv', delim_whitespace=False)

#Format datetime:
River['datetime'] = River['Activity_StartDate']
River['datetime'] = River['datetime'].astype('datetime64[ns]')

In [ ]:
#If needed, separate the suspended sediment data and the water discharge data: 
River_SSC = River[(River['Result_Characteristic'] == 'Suspended Sediment Concentration (SSC)')
River_Q = River[River['Result_Characteristic'] == 'Stream flow, instantaneous']

In [ ]:
# Merge River_Q and River_SSC on the 'datetime' column to find matching dates. 
River_Combined = pd.merge(River_Q, River_SSC, on='datetime')

#Plot SSC vs Q:
fig, ax1 = plt.subplots(figsize=(5,5), sharex=True)
ax1.scatter(River_Combined['Result_Measure_x'], River_Combined['Result_Measure_y'])           
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_title('River SSC vs Q')
ax1.set_xlabel('Q (m/s)')
ax1.set_ylabel('SSC (mg/L)')

#Create rating curve using the eqaution a*Q**b: 
def power_law(Q_River, a_River, b_River):
    return a_River*Q_River**b_River

#popt contains optimal values for best fitting the data, pcov estimates uncertainty.
popt, pcov = curve_fit(power_law, Q_River, River_Combined['Result_Measure_y'])
a_River, b_River = popt

print(f"River Estimated a: {a_River}")
print(f"River Estimated b: {b_River}")

#Plot trendline based on found values for "a" and "b":
y_fit = power_law(Q_River, *popt)
ax1.plot(Q_River, y_fit, label=f'Fit: a={a_River:.3f}, b={b_River:.3f}', color='blue')
ax1.legend()